From the main directory "Ontolearn", run the commands for NCES data mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download pretrained models and datasets.

## Inference with NCES

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.learning_problem import PosNegLPStandard
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.metrics import F1
from owlapy.parser import DLSyntaxParser

/opt/conda/envs/onto_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", quality_func=F1(), num_predictions=100, learner_names=["SetTransformer"],
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32)


 Loaded NCES weights!



In [3]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

### Let's learn the concept ``Father''

In [5]:
father = dl_parser.parse('Father')

In [6]:
not_father = dl_parser.parse('¬Father') # For negative examples

In [7]:
pos = set([ind.str.split("/")[-1] for ind in KB.individuals(father)])

In [8]:
neg = set([ind.str.split("/")[-1] for ind in KB.individuals(not_father)])

#### Prediction with SetTransformer

In [9]:
lp = PosNegLPStandard(pos=pos, neg=neg)
nodes = nces.fit(lp).best_hypotheses(3)

In [10]:
nodes

[OWLObjectIntersectionOf((OWLClass(IRI('http://www.benchmark.org/family#','Father')), OWLObjectUnionOf((OWLClass(IRI('http://www.benchmark.org/family#','Father')), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasParent')),filler=OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling'))))), OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasParent')),filler=OWLClass(IRI('http://www.w3.org/2002/07/owl#','Thing'))))),
 OWLObjectUnionOf((OWLClass(IRI('http://www.benchmark.org/family#','Father')), OWLClass(IRI('http://www.benchmark.org/family#','Father')), OWLClass(IRI('http://www.benchmark.org/family#','Grandfather')), OWLClass(IRI('http://www.benchmark.org/family#','Grandfather')))),
 OWLClass(IRI('http://www.benchmark.org/family#','Father'))]

In [11]:
nces.sorted_examples

False

#### Prediction with GRU

In [12]:
nces.learner_names = ['GRU']
nces.refresh()
nodes = list(nces.fit(lp).best_predictions)


 Loaded NCES weights!



In [13]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x69d8928	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x70389f8	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7038a54	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69d8884	Grandfather ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69db160	Father ⊔ Grandfather	Quality:1.0	Length:3	|Indv.|:60]

#### Prediction with LSTM

In [14]:
nces.learner_names = ['LSTM']
nces.refresh()
nodes = list(nces.fit(lp).best_predictions)


 Loaded NCES weights!



In [15]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x69da730	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7035258	Male ⊓ (∃ hasChild.(Female ⊔ (∃ hasParent.(¬Daughter))))	Quality:1.0	Length:10	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69bcd10	Father ⊓ (Father ⊓ (Male ⊔ (∃ hasSibling.(¬Grandfather))))	Quality:1.0	Length:10	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69d93b0	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7038a78	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60]

#### Prediction with ensemble SetTransformer+GRU+LSTM

In [16]:
nces.learner_names = ['SetTransformer','GRU','LSTM']
nces.refresh() # Load model weights
nodes = list(nces.fit(lp).best_predictions)


 Loaded NCES weights!



In [17]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x69d8970	Father ⊔ Grandfather	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69d889c	Father ⊔ Grandfather	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x702cc18	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69d7358	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x69d7274	Father	Quality:1.0	Length:1	|Indv.|:60]

### Scalability of NCES (solving multiple learning problems in a go!)

In [18]:
import json, time

In [19]:
with open("../NCESData/family/training_data/Data.json") as file:
    data = list(json.load(file).items())
## The function below takes an iterable of tuples
LPs = list(map(lambda x: (x[0], x[1]["positive examples"], x[1]["negative examples"]), data))

In [20]:
## We solve 256 learning problems!
lps = LPs[:256]

In [21]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False) ## Now predict with nces.fit_from_iterable.
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  0.46558618545532227 seconds!


In [22]:
print(concepts[0])

OWLClass(IRI('http://www.benchmark.org/family#','Sister'))


In [23]:
print(len(concepts))

256


### Change pretrained model name, e.g., use ensemble model prediction

In [24]:
nces.learner_names = ['SetTransformer', 'GRU']

In [25]:
nces.learner_names

['SetTransformer', 'GRU']

In [26]:
nces.refresh()


 Loaded NCES weights!



In [27]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False)
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  0.8283030986785889 seconds!


In [28]:
concepts[-1]

OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling'))))

## Training NCES

In [29]:
import json, time
with open("../NCESData/family/training_data/Data.json") as file:
    data = json.load(file) # Training data. Below we use the first 200 datapoints to train the synthesizer. You train on the full data, e.g. on GPU by remobing "[:200]"

In [30]:
from ontolearn.concept_learner import NCES

In [31]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_names=["SetTransformer"],
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,
            load_pretrained=False)

!!!Returning untrained models, could not load pretrained. Check the `load_pretrained parameter` or train the models using NCES.train(data).


In [32]:
nces.train(list(data.items())[-200:], epochs=20, learning_rate=0.001, save_model=True)

********************Trainable model size********************
Synthesizer:  515296
********************Trainable model size********************

GPU available !

##################################################

SetTransformer starts training... 

################################################## 



Loss: 0.4141, Soft Acc: 72.98%, Hard Acc: 72.07%: 100%|██████████| 20/20 [00:05<00:00,  3.43it/s]

Top performance: loss: 0.4141, soft accuracy: 77.84% ... hard accuracy: 76.40%

SetTransformer saved
